In [19]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


# Task: Columns -> Energy & Petrol Consumption, Storage

In [20]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan", "Tesla"]

def get_data(base, maker):
    mpge = []
    mpg = []
    battery = []
    gas_tank = []
    links = []
    
    manufacturers = []
    years = []
    models = []
    miles_IRA = [] 

    emissions = []
    types = []
    min_prices = []
    max_prices = []
    
    for m in maker:
        page = 1
        while True:
            print(f"Scraping page {page} of {m}")
            maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={m}&srchtyp=newMake&pageno={page}&rowLimit=200"
            response = r.get(maker_url)
            soup = bs(response.content)
            rows = soup.find_all("tr", class_ = "ymm-row")
            if not rows:
                break

            n = 2
            for tr in rows:

                a_tags = tr.find_all("a")
                for a in a_tags:
                    manufacturers.append(a.text.split()[1])
                    years.append(a.text.split()[0])
                    models.append(' '.join(a.text.split()[2:]))   

                miles_IRA.extend([mile_IRA.get_text() for mile_IRA in soup.find_all("div", class_="totalRange")])
            
                emiss = soup.select(f".cars > tbody:nth-child(2) > tr:nth-child({n}) > td:nth-child(8) > div:nth-child(2)")
                price = soup.select(f".cars > tbody:nth-child(2) > tr:nth-child({n}) > td:nth-child(6)")
                try:
                    emission_value = emiss[0].get_text(strip=True).split()[0]
                    price_text = price[0].get_text(strip=True)
                    cleaned_price = price_text.replace('$', '').replace(',', '')  # Remove $ and comma
                    price_parts = cleaned_price.split('–')
                    price_parts = [part.strip() for part in price_parts if part]
                    # print("Emission: ",emission_value, "Number: ",n)
                    n += 3
                    emissions.append(emission_value)
                    if len(price_parts) == 1:
                        min_price = float(price_parts[0])
                        max_price = float(price_parts[0])
                    elif len(price_parts) == 2:
                        min_price = float(price_parts[0])
                        max_price = float(price_parts[1])
                    else:
                        min_price = None
                        max_price = None
    
                    min_prices.append(min_price)
                    max_prices.append(max_price)
                except:
                    n += 2
                    emiss = soup.select(f".cars > tbody:nth-child(2) > tr:nth-child({n}) > td:nth-child(8) > div:nth-child(2)")
                    price = soup.select(f".cars > tbody:nth-child(2) > tr:nth-child({n}) > td:nth-child(6)") 
                    try:
                        emission_value = emiss[0].get_text(strip=True).split()[0]
                        price_text = price[0].get_text(strip=True)
                        cleaned_price = price_text.replace('$', '').replace(',', '')  # Remove $ and comma
                        price_parts = cleaned_price.split('–')
                        price_parts = [part.strip() for part in price_parts if part]
                    except:
                        pass
                    else:
                        n += 3
                    # print("Emission: ",emission_value, "New Number: ",n)
                    emissions.append(emission_value)
                    if len(price_parts) == 1:
                        min_price = float(price_parts[0])
                        max_price = float(price_parts[0])
                    elif len(price_parts) == 2:
                        min_price = float(price_parts[0])
                        max_price = float(price_parts[1])
                    else:
                        min_price = None
                        max_price = None
    
                    min_prices.append(min_price)
                    max_prices.append(max_price)

            config_span = soup.find_all("span", class_="config")
            for e in config_span:
                car_type_text = e.get_text(strip=True) if config_span else None
                if "Electricity" in car_type_text:
                    car_type = "Electricity"
                elif "Regular Gasoline" in car_type_text:
                    car_type = "Regular Gasoline"
                elif "Diesel" in car_type_text:
                    car_type = "Diesel"
                elif "Premium Gasoline" in car_type_text:
                    car_type = "Premium Gasoline"
                elif "E85" in car_type_text:
                    car_type = "E85"
                else:
                    car_type = "Unknown"                
                types.append(car_type)

            link = [x.get("href") for x in soup.select("a[href]") if "sbs&id" in x.get("href")]

            for l in link:
                if l == "#":
                    continue
                links.append(l)
                response = r.get(base+l)
                soup = bs(response.content)
                try:
                    type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
                except:
                    print(f"No type: {l}")
                try:
                    if type == "Electricity":
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                        gas_tank.append(0)
                        mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                        mpg.append(0)
                    elif type in ["Regular Gasoline", "Diesel", "Premium Gasoline"]:
                        battery.append(0)
                        if soup.find_all("td", class_="sbsEconData")[2].getText().split(" ")[0] == "E85:":
                            gas_tank.append(soup.find_all("td", class_="sbsEconData")[5].getText().split(" ")[0])
                        else:
                            gas_tank.append(soup.find_all("td", class_="sbsEconData")[2].getText().split(" ")[0]) #gallons
                        mpge.append(0)
                        mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                    else:
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                        gas_tank.append(soup.find_all("td", class_="sbsEconData")[3].getText().split(" ")[0]) #gallons
                        mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                        mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                except:
                    print(f"Process stopped at {l}")
            page += 1
            
    miles_IRA = miles_IRA[:len(manufacturers)] 
    
    print("Creating Dictionary.")
    print("\n")

    manufacturer_dict = {"manufacturer":manufacturers, "year":years, "model":models, "total_range_in_mls":miles_IRA,
                         'emission_value':emissions, 'min_price':min_prices, 'max_price':max_prices, 'car_type':types,
                         "electric_consumption":mpge, "electric_storage":battery, 
                         "analogue_consumption":mpg, "analogue_storage":gas_tank, "links":links}
    print("\n")
    print("Checking Dataframe.")
    for x,y in manufacturer_dict.items():
        print(f"{x}: {len(y)}")
    print("\nCreating Dataframe.")
    return pd.DataFrame(manufacturer_dict)

manufacturer_df = get_data(base_url, makers)
print("Finished")

Scraping page 1 of Chevrolet
Scraping page 2 of Chevrolet
Scraping page 3 of Chevrolet
Scraping page 4 of Chevrolet
Scraping page 1 of Ford
Scraping page 2 of Ford
Scraping page 3 of Ford
Scraping page 4 of Ford
Scraping page 1 of Nissan
Scraping page 2 of Nissan
Scraping page 3 of Nissan
Scraping page 1 of Tesla
Scraping page 2 of Tesla
Creating Dictionary.




Checking Dataframe.
manufacturer: 1366
year: 1366
model: 1366
total_range_in_mls: 1366
emission_value: 1366
min_price: 1366
max_price: 1366
car_type: 1366
electric_consumption: 1366
electric_storage: 1366
analogue_consumption: 1366
analogue_storage: 1366
links: 1366

Creating Dataframe.
Finished


In [25]:
manufacturer_df_copy = manufacturer_df

In [34]:
manufacturer_df_copy["analogue_storage"] = manufacturer_df_copy["analogue_storage"].astype(str).apply(lambda x: (int(float(x.split("-")[0]))+
                                                                                                                 int(float(x.split("-")[1])))/2
                                                                                                      if "-" in x else x)
manufacturer_df_copy["analogue_storage"] = manufacturer_df_copy["analogue_storage"].apply(lambda x: 0 if not x else x)
manufacturer_df_copy['year'] = manufacturer_df_copy['year'].astype(int)
# manufacturer_df_copy['total_range_in_mls'] = manufacturer_df_copy['total_range_in_mls'].str.split().str[0].astype(int)

In [35]:
manufacturer_df_copy

manufacturer  year                              model  \
0       Chevrolet  2023                            Bolt EV   
1       Chevrolet  2022                            Bolt EV   
2       Chevrolet  2019                            Bolt EV   
3       Chevrolet  2021                            Bolt EV   
4       Chevrolet  2020                            Bolt EV   
...           ...   ...                                ...   
1361        Tesla  2019                       Model X 100D   
1362        Tesla  2021  Model X Performance (22in Wheels)   
1363        Tesla  2019                      Model X P100D   
1364        Tesla  2020  Model X Performance (22in Wheels)   
1365        Tesla  2019  Model X Performance (22in Wheels)   

      total_range_in_mls emission_value  min_price  max_price     car_type  \
0                    259              0    26500.0    29700.0  Electricity   
1                    259              0    31000.0    34200.0  Electricity   
2                    238              0    36620.0    41020.0  Electricity   
3                    259              0    36500.0    41700.0  Electricity   
4                    259              0    36620.0    41020.0  Electricity   
...                  ...            ...        ...        ...          ...   
1361                 598              0    97000.0    97000.0  Electricity   
1362                 500              0        NaN        NaN  Electricity   
1363                 472              0   138000.0   138000.0  Electricity   
1364                 452              0    99990.0    99990.0  Electricity   
1365                 297              0   104990.0   104990.0  Electricity   

     electric_consumption electric_storage analogue_consumption  \
0                     120              7.5                    0   
1                     120              7.5                    0   
2                     119              9.3                    0   
3                     118               10                    0   
4                     118               10                    0   
...                   ...              ...                  ...   
1361                   87               12                    0   
1362                   86               15                    0   
1363                   85               12                    0   
1364                   79               12                    0   
1365                   79               12                    0   

     analogue_storage                        links  
0                   0  Find.do?action=sbs&id=45751  
1                   0  Find.do?action=sbs&id=43955  
2                   0  Find.do?action=sbs&id=40520  
3                   0  Find.do?action=sbs&id=43663  
4                   0  Find.do?action=sbs&id=42191  
...               ...                          ...  
1361                0  Find.do?action=sbs&id=41196  
1362                0  Find.do?action=sbs&id=43405  
1363                0  Find.do?action=sbs&id=41197  
1364                0  Find.do?action=sbs&id=42288  
1365                0  Find.do?action=sbs&id=41515  

[1366 rows x 13 columns]

In [36]:
manufacturer_df_copy.isna().sum()
manufacturer_df_copy.duplicated().sum()
manufacturer_df_copy.fillna(0)
manufacturer_df_copy.drop_duplicates()

manufacturer_df_copy.to_csv('finished_df_full.csv', index=False) 

In [ ]:
s_df = pd.read_csv("sergej_df.csv")
m_df = pd.read_csv("mateo_df.csv")
r_df = pd.read_csv("rosi_df.csv")

In [ ]:
all_df = pd.concat([m_df,s_df,r_df], axis=1)

In [ ]:
all_df.head(3)

In [ ]:
all_df.isna().sum()
all_df.duplicated().sum()
all_df.fillna(0)
all_df.drop_duplicates()

In [ ]:
all_df

In [ ]:
all_df.groupby(["car_type"])["total_range_in_mls"].mean().sort_values(ascending=False)

In [ ]:
all_df.groupby(["car_type"])["max_price"].mean().sort_values(ascending=False)

In [ ]:
all_df.groupby(["car_type"])["emission_value"].mean().sort_values(ascending=False)

In [ ]:
range_comparison = all_df.pivot_table(values="total_range_in_mls", index="manufacturer", columns="car_type",
                                         aggfunc="mean")
range_comparison

In [ ]:
all_df["car_type"].value_counts()